# **SML Project: Binary Tree Predictors**

*   **Author:** Matteo Onger
*   **Date:** October 2024

**Dataset documentation**:
*   [Secondary Mushroom](https://archive.ics.uci.edu/dataset/848/secondary+mushroom+dataset)

## VM Setup

In [25]:
# install dataset package
!pip install ucimlrepo

# download repository
!git clone -b dev https://github.com/MatteoOnger/SML_Project.git

# set working directory
%cd /content/SML_Project/

fatal: destination path 'SML_Project' already exists and is not an empty directory.
/content/SML_Project


## Code

In [26]:
# ---- LIBRARIES ----
import logging
import pandas as pd

from sklearn.model_selection import KFold
from typing import Any, Dict, List, Tuple, Type
from ucimlrepo import fetch_ucirepo

from binrandomforest import BinRandomForest
from bintreepredictor import BinTreePredictor
from data import DataSet
from utils import round_wrp

In [27]:
# chose logging level between WARNING, INFO or DEBUG
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(name)s - %(message)s", force=True)

In [28]:
# ---- FUNCTIONS ----
def k_folds_cross_val(
        k :int,
        predictor :BinRandomForest|BinTreePredictor,
        data :pd.DataFrame,
        label_col :str,
        shuffle :bool=True,
        random_state :int=1,
        verbose :bool=False
    ) -> Tuple[float, float]:
    """
    Applies the k-folds cross validation to estimate the expected risk of the predictor.

    Parameters
    ----------
    k : int
        Number of folds.
    predictor : BinRandomForest | BinTreePredictor
        Predictor that must be tested.
    data : pd.DataFrame
        Data used to train and test the predictor.
    label_col : str
        The name of the column of ``data`` that contains the labels.
    shuffle : bool, optional
        Whether to shuffle the data before splitting into batches, by default True.
    random_state : int, optional
        When shuffle is True, random_state affects the ordering of the indices, which controls the randomness of each fold.
        Otherwise, this parameter has no effect, by default 1.
    verbose : bool, optional
        If True, training and test error of each fold are printed, by default False.

    Returns
    -------
    Tuple[float, float]
        The tuple returned contains the average training error and the average test error.
    """
    avg_train_err = 0
    avg_test_err = 0

    cv = KFold(n_splits=k, shuffle=shuffle, random_state=random_state)

    for i, (train_index, test_index) in enumerate(cv.split(data)):
        train_ds = DataSet(data.iloc[train_index], label_col=label_col)
        test_ds =  DataSet(data.iloc[test_index], label_col=label_col)

        train_err = predictor.fit(train_ds)
        _, test_err = predictor.predict(test_ds)

        if verbose:
            print(f"round {i} - training_err:{round_wrp(train_err,4)} - test_err:{round_wrp(test_err,4)}")

        avg_train_err += train_err
        avg_test_err += test_err
    return avg_train_err / k, avg_test_err / k


def nested_cross_val(
        outer_k :int,
        inner_k :int,
        predicotr_class :Type[BinTreePredictor]|Type[BinRandomForest],
        fixed_hyperparams :Dict[str, Any],
        hyperparams :List[Dict[str, Any]],
        data :pd.DataFrame,
        label_col :str,
        shuffle :bool=True,
        random_state :int=1,
        verbose :bool=False
    ) -> Tuple[float, float]:
    """
    Applies the nested cross validation to estimate the expected risk of the learning algorithm.

    Parameters
    ----------
    outer_k : int
        Number of folds of the outer CV.
    inner_k : int
        Number of folds of the inner CV.
    predicotr_class : Type[BinTreePredictor] | Type[BinRandomForest]
        Class of the predictor that must be tested.
    fixed_hyperparams : Dict[str, Any]
        Fixed hyper-parameters on which no tuning is performed.
    hyperparams : List[Dict[str, Any]]
        Hyper-parameters to be tuned.
        A list containing all the combinations of the hyper-parameters to try must be given, then the best combination will be used.
    data : pd.DataFrame
        Data used to train and test the predictor.
    label_col : str
        The name of the column of ``data`` that contains the labels.
    shuffle : bool, optional
        Whether to shuffle the data before splitting into batches, by default True.
    random_state : int, optional
        When shuffle is True, random_state affects the ordering of the indices, which controls the randomness of each fold.
        Otherwise, this parameter has no effect, by default 1.
    verbose : bool, optional
        If True, training, validation and test error of each fold are printed, by default False.

    Returns
    -------
    Tuple[float, float]
        The tuple returned contains the average training error and the average test error.
    """
    avg_train_err = 0
    avg_test_err = 0

    outer_cv = KFold(n_splits=outer_k, shuffle=shuffle, random_state=random_state)

    for i, (train_index, test_index) in enumerate(outer_cv.split(data)):
        train_df = data.iloc[train_index]
        test_df = data.iloc[test_index]

        train_ds = DataSet(train_df, label_col=label_col)
        test_ds = DataSet(test_df, label_col=label_col)

        best_hyperparam, best_val_err = None, float("inf")

        for j, hp in enumerate(hyperparams):
            predictor = predicotr_class(**fixed_hyperparams, **hp, id=j)
            _, avg_val_err = k_folds_cross_val(inner_k, predictor, train_df, label_col, shuffle, random_state)

            if avg_val_err < best_val_err:
                best_val_err = avg_val_err
                best_hyperparam = hp

        predictor = predicotr_class(**fixed_hyperparams, **best_hyperparam, id=-i)
        train_err = predictor.fit(train_ds)
        _, test_err = predictor.predict(test_ds)

        if verbose:
            print(f"round {i} - training_err:{round_wrp(train_err,4)} - validation_err:{round_wrp(best_val_err,4)} - test_err:{round_wrp(test_err,4)}")

        avg_train_err += train_err
        avg_test_err += test_err
    return avg_train_err / outer_k, avg_test_err / outer_k


def eval_hyperparams(
        k :int,
        predicotr_class :Type[BinTreePredictor]|Type[BinRandomForest],
        fixed_hyperparams :Dict[str, Any],
        hyperparams :List[Dict[str, Any]],
        data :pd.DataFrame,
        label_col :str,
        shuffle :bool=True,
        random_state :int=1,
        verbose :bool=False
    ) -> Dict[int, Dict[str, float]]:
    """
    Trains one predictor for each combination of the hyper-parameters given in ``hyperparams`` and then
    applies the k-folds cross validation to estimate its expected risk.

    Parameters
    ----------
    k : int
        Number of folds.
    predicotr_class : Type[BinTreePredictor] | Type[BinRandomForest]
        Class of the predictor that must be tested.
    fixed_hyperparams : Dict[str, Any]
        Fixed hyperparameters.
    hyperparams : List[Dict[str, Any]]
        A list containing all the combinations of the hyperparameters to try.
    data : pd.DataFrame
        Data used to train and test the predictor.
    label_col : str
        The name of the column of ``data`` that contains the labels.
    shuffle : bool, optional
        Whether to shuffle the data before splitting into batches, by default True.
    random_state : int, optional
        When shuffle is True, random_state affects the ordering of the indices, which controls the randomness of each fold.
        Otherwise, this parameter has no effect, by default 1.
    verbose : bool, optional
        If True, training and test error of each fold are printed, by default False.

    Returns
    -------
    Dict[int, Dict[str, float]]
        The average trainig and test error of each predictor produced.
    """
    results = dict()
    for i, hp in enumerate(hyperparams):
        predictor = predicotr_class(**fixed_hyperparams, **hp, id=i)
        avg_train_err, avg_test_err = k_folds_cross_val(k, predictor, data, label_col, shuffle, random_state, verbose)
        results[i] = {"avg_train_err":avg_train_err, "avg_test_err":avg_test_err}
    return results

### Dataset

In [29]:
# fetch datatset
mushroom_df = fetch_ucirepo(id=848).data.original
mushroom_df.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w


### Binary Tree Predictors

In [31]:
# use k-folds CV to estimate the expected risk of the predictors produced by setting to different values the hyper-parameters
k = 5

fixed_hyperparams = {
    "loss_func":"zero-one",
    "prediction_criterion":"mode",
    "split_criterion":"entropy",
    "stop_criterion":"max_nodes",
    "max_features":None,
    "max_thresholds":5,
}

hyperparams = [
    {"stop_criterion_threshold": 32},
    {"stop_criterion_threshold": 64},
    {"stop_criterion_threshold":128},
]

eval_hyperparams(k, BinTreePredictor, fixed_hyperparams, hyperparams, mushroom_df, "class")

2024-10-14 10:12:23,878 - INFO - bintreepredictor - BinTreePredictor_id:0 - training_err:0.2441
2024-10-14 10:12:23,978 - INFO - bintreepredictor - BinTreePredictor_id:0 - test_err:0.2414
2024-10-14 10:13:33,102 - INFO - bintreepredictor - BinTreePredictor_id:0 - training_err:0.2351
2024-10-14 10:13:33,177 - INFO - bintreepredictor - BinTreePredictor_id:0 - test_err:0.2431
2024-10-14 10:14:41,036 - INFO - bintreepredictor - BinTreePredictor_id:0 - training_err:0.2501
2024-10-14 10:14:41,146 - INFO - bintreepredictor - BinTreePredictor_id:0 - test_err:0.25
2024-10-14 10:15:48,894 - INFO - bintreepredictor - BinTreePredictor_id:0 - training_err:0.25
2024-10-14 10:15:48,961 - INFO - bintreepredictor - BinTreePredictor_id:0 - test_err:0.2508
2024-10-14 10:16:57,973 - INFO - bintreepredictor - BinTreePredictor_id:0 - training_err:0.244
2024-10-14 10:16:58,070 - INFO - bintreepredictor - BinTreePredictor_id:0 - test_err:0.2417
2024-10-14 10:19:16,410 - INFO - bintreepredictor - BinTreePredic

{0: {'avg_train_err': 0.24467405979441073,
  'avg_test_err': 0.24539449336259453},
 1: {'avg_train_err': 0.16284860240727664,
  'avg_test_err': 0.16409295428608225},
 2: {'avg_train_err': 0.05858541541198472,
  'avg_test_err': 0.06011214672439049}}

In [ ]:
# nested cross validation to obtain a more accurate estimate of the performance of the learning algorithm.
inner_k = 5
outer_k = 10

fixed_hyperparams = {
    "loss_func":"zero-one",
    "prediction_criterion":"mode",
    "stop_criterion":"max_height",
    "max_features":None,
    "max_thresholds":5,
}

hyperparams = [
    {"split_criterion":"entropy", "stop_criterion_threshold": 15},
    {"split_criterion":"entropy", "stop_criterion_threshold": 20},
    {"split_criterion":"gini", "stop_criterion_threshold": 15},
    {"split_criterion":"gini", "stop_criterion_threshold": 20},
]

nested_cross_val(outer_k, inner_k, BinTreePredictor, fixed_hyperparams, hyperparams, mushroom_df, "class", verbose=True)

In [ ]:
# train the predictor, test the performance and print the results
mushroom_ds = DataSet(mushroom_df, label_col="class")

train_ds = mushroom_ds.sample(frac=0.8, seed=2106)
test_ds = mushroom_ds.drop(train_ds.index)

tree = BinTreePredictor("zero-one", "mode", "gini", "max_height", 20, max_thresholds=5)
train_err = tree.fit(train_ds)
prediction, test_err = tree.predict(test_ds)

tree.print_tree()
print(f"Training error:{round_wrp(train_err,5)}")
print(f"Test error:{round(test_err,5)}")

test_ds.insert(1, "predicted-label", prediction).data

2024-10-14 08:32:11,837 - INFO - bintreepredictor - BinTreePredictor_id:0 - training_err:0.0013
2024-10-14 08:32:12,098 - INFO - bintreepredictor - BinTreePredictor_id:0 - test_err:0.0024



---- TREE ----
tree -> {'id': 0, 'prediction_criterion': 'mode', 'split_criterion': 'gini', 'stop_criterion': '(max_height, 20)', 'num_nodes': 259, 'height': 21, 'max_features': None, 'max_thresholds': 5, 'root': 0, 'num_leaves': 130}

node -> {'id': 0, 'type': 'inner', 'tree_id': 0, 'depth': 0, 'parent_id': None, 'sx_id': 1, 'dx_id': 2, 'test': '(stem-color, w)', 'prediction': None, 'has_data': None, 'is_pure': None}
node -> {'id': 1, 'type': 'inner', 'tree_id': 0, 'depth': 1, 'parent_id': 0, 'sx_id': 3, 'dx_id': 4, 'test': '(cap-shape, b)', 'prediction': None, 'has_data': None, 'is_pure': None}
node -> {'id': 2, 'type': 'inner', 'tree_id': 0, 'depth': 1, 'parent_id': 0, 'sx_id': 5, 'dx_id': 6, 'test': '(gill-attachment, p)', 'prediction': None, 'has_data': None, 'is_pure': None}
node -> {'id': 3, 'type': 'inner', 'tree_id': 0, 'depth': 2, 'parent_id': 1, 'sx_id': 7, 'dx_id': 8, 'test': '(stem-width, 4.2317)', 'prediction': None, 'has_data': None, 'is_pure': None}
node -> {'id': 4, '

,class,predicted-label,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
9,p,p,13.55,f,g,e,f,e,NaN,w,...,s,y,w,u,w,t,p,NaN,d,w
17,p,p,17.40,x,h,o,f,e,NaN,w,...,s,y,w,u,w,t,p,NaN,d,a
21,p,p,13.06,x,g,e,f,e,NaN,w,...,s,y,w,u,w,t,g,NaN,d,u
22,p,p,17.23,x,h,o,f,e,NaN,w,...,s,y,w,u,w,t,g,NaN,d,w
30,p,p,16.58,x,g,o,f,e,NaN,w,...,s,y,w,u,w,t,g,NaN,d,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61049,p,p,1.16,x,s,y,f,f,f,f,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a
61052,p,p,1.30,f,s,y,f,f,f,f,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,u
61063,p,p,1.42,x,s,y,f,f,f,f,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a
61064,p,p,1.18,s,s,y,f,f,f,f,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a


### Binary Random Forest

In [ ]:
# use k-folds CV to estimate the expected risk of the predictors produced by setting to different values the hyper-parameters
k = 5

fixed_hyperparams = {
    "loss_func":"zero-one",
    "prediction_criterion":"mode",
    "split_criterion":"gini",
    "stop_criterion":"max_height",
    "stop_criterion_threshold": 15,
    "max_features":"sqrt",
    "max_thresholds":5,
}

hyperparams = [
    {"num_trees":8},
    {"num_trees":16},
    {"num_trees":32},
    {"num_trees":64},
]

eval_hyperparams(k, BinRandomForest, fixed_hyperparams, hyperparams, mushroom_df, "class")

In [ ]:
# train the predictor, test the performance and print the results
mushroom_ds = DataSet(mushroom_df, label_col="class")

train_ds = mushroom_ds.sample(frac=0.8, seed=2106)
test_ds = mushroom_ds.drop(train_ds.index)

forest = BinRandomForest(8, "zero-one", "mode", "gini", "max_nodes", 64, "sqrt", max_thresholds=5)
train_err = forest.fit(train_ds)
prediction, test_err = forest.predict(test_ds)

print(forest)
print(f"Training error:{round_wrp(train_err,5)}")
print(f"Test error:{round(test_err,5)}")

test_ds.insert(1, "predicted-label", prediction).data

2024-10-14 08:37:00,413 - INFO - bintreepredictor - BinTreePredictor_id:0 - training_err:0.1545
2024-10-14 08:37:41,125 - INFO - bintreepredictor - BinTreePredictor_id:1 - training_err:0.164
2024-10-14 08:38:30,857 - INFO - bintreepredictor - BinTreePredictor_id:2 - training_err:0.1482
2024-10-14 08:39:13,106 - INFO - bintreepredictor - BinTreePredictor_id:3 - training_err:0.1701
2024-10-14 08:40:00,090 - INFO - bintreepredictor - BinTreePredictor_id:4 - training_err:0.1417
2024-10-14 08:40:43,575 - INFO - bintreepredictor - BinTreePredictor_id:5 - training_err:0.1222
2024-10-14 08:41:28,267 - INFO - bintreepredictor - BinTreePredictor_id:6 - training_err:0.1506
2024-10-14 08:42:12,987 - INFO - bintreepredictor - BinTreePredictor_id:7 - training_err:0.1389
2024-10-14 08:42:13,237 - INFO - bintreepredictor - BinTreePredictor_id:0 - test_err:0.1527
2024-10-14 08:42:13,476 - INFO - bintreepredictor - BinTreePredictor_id:1 - test_err:0.1646
2024-10-14 08:42:13,708 - INFO - bintreepredictor

random_forest -> {'id': 0, 'num_trees': 8, 'prediction_criterion': 'mode', 'split_criterion': 'gini', 'stop_criterion': '(max_nodes, 64)', 'max_features': 5, 'max_thresholds': 5}
Training error:0.10971
Test error:0.10488


,class,predicted-label,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
9,p,p,13.55,f,g,e,f,e,NaN,w,...,s,y,w,u,w,t,p,NaN,d,w
17,p,e,17.40,x,h,o,f,e,NaN,w,...,s,y,w,u,w,t,p,NaN,d,a
21,p,p,13.06,x,g,e,f,e,NaN,w,...,s,y,w,u,w,t,g,NaN,d,u
22,p,e,17.23,x,h,o,f,e,NaN,w,...,s,y,w,u,w,t,g,NaN,d,w
30,p,p,16.58,x,g,o,f,e,NaN,w,...,s,y,w,u,w,t,g,NaN,d,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61049,p,p,1.16,x,s,y,f,f,f,f,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a
61052,p,p,1.30,f,s,y,f,f,f,f,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,u
61063,p,e,1.42,x,s,y,f,f,f,f,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a
61064,p,p,1.18,s,s,y,f,f,f,f,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a
